In [13]:
import pandas as pd
import re

from selenium import webdriver
from time import sleep
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains  # 用于鼠标悬停
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

import matplotlib.pyplot as plt
%matplotlib inline


In [14]:
# 启动 Edge 浏览器
driver = webdriver.Edge()

# 访问二手房首页
url = 'https://esf.fang.com/'
driver.get(url)

# 等待页面加载
sleep(5)

In [17]:
# 找到地区选择的按钮，点击天津
hover_element = driver.find_element(By.CSS_SELECTOR, "div.s4Box")

actions = ActionChains(driver)
actions.move_to_element(hover_element).perform()

tianjin_city = driver.find_element(By.XPATH, "//div[@class='city20141104nr' and @id='cityi010']/a[text()='天津']")
tianjin_city.click()

sleep(5)

In [20]:
# 找到“滨海新区”对应的链接
binhai = driver.find_element(By.LINK_TEXT, "滨海新区")
binhai.click()

sleep(5)

In [21]:
# 找到“中新生态城”对应的链接
zx_city = driver.find_element(By.LINK_TEXT, "中新生态城")
zx_city.click()

sleep(5)

In [22]:
from selenium.common.exceptions import NoSuchElementException

house_data = []  # 房源数据汇总

while True:
    # 1. 找到房源列表并解析当前页数据
    try:
        shop_list_container = driver.find_element(By.CSS_SELECTOR, "div.shop_list.shop_list_4") #整个表格
    except NoSuchElementException:
        print("未找到房源列表")
        break  # 若连列表都无法找到，直接退出循环
    
    # 找到每一条房源的dl标签
    dl_elements = shop_list_container.find_elements(By.TAG_NAME, "dl")

    for dl in dl_elements:
        try:
            # 标题信息
            title = dl.find_element(By.XPATH, "./dd/h4/a/span").text.strip()

            # 户型 & 面积
            house_info = dl.find_element(By.XPATH, "./dd/p[@class='tel_shop']").text.strip()
            # 例如 "2室2厅 | 109.07㎡ | 高层 （共9层） | 南北向 | 2006年建 | 中介姓名"

            # 价格
            price_total = dl.find_element(By.XPATH, "./dd[@class='price_right']/span[1]").text.strip()  # 总价
            price_unit  = dl.find_element(By.XPATH, "./dd[@class='price_right']/span[2]").text.strip()   # 单价

            # 存储数据到列表
            house_data.append({
                "标题": title,
                "户型_面积等": house_info,
                "总价": price_total,
                "单价": price_unit
            })
        except NoSuchElementException:
            # 有些dl可能是广告位或缺少字段，直接忽略
            continue

    #  2. 找到“下一页”并点击翻页，直到找不到结束 
    try:
        sleep(2)
        # 下一页按钮
        next_page = driver.find_element(By.LINK_TEXT, "下一页")
        next_page.click()
        # 等待
        sleep(3)
    except NoSuchElementException:
        print("没有找到下一页按钮, 爬取结束!")
        break

# 全部页面爬取结束后，将 house_data 转为 DataFrame
df = pd.DataFrame(house_data)
df


没有找到下一页按钮, 爬取结束!


,标题,户型_面积等,总价,单价
0,好房推荐80万宝龙城北苑89.5平价格真实机会难得快快快快,3室2厅 | 89.5㎡ |低层 （共11层） | 南北向 | 2017年建 |张伟,80万,8938元/㎡
1,生态城中部可以上南开片 贻成精装修中间楼层全阳户型业主急售,2室2厅 | 83㎡ |低层 （共28层） | 南北向 | 2020年建 |张伟,120万,14457元/㎡
2,新出工抵房6套。生态城博屿海 南北通透三居室户型。近,3室2厅 | 103㎡ | 底层 （共18层） | 南北向 | 2018年建 |张伟,76万,7378元/㎡
3,工抵房6套66万精装小高洋房总价低近车接车送靠谱,3室2厅 | 87㎡ |低层 （共9层） | 南北向 | 2006年建 |王飞,65万,7471元/㎡
4,南北通透 小三房 装修好 集中供暖 临地铁 满五,3室2厅 | 90㎡ |中层 （共11层） | 南北向 | 2017年建 |李嘉,85万,9444元/㎡
...,...,...,...,...
1553,生态城万达旁精装红橡美韵园 3室2厅 送车位送家具,3室2厅 | 98㎡ |高层 （共16层） | 南向,156万,15918元/㎡
1554,滨海新区 中加生态示范区 3室2厅,3室2厅 | 117㎡ |中层 （共8层） | 南北向,180万,15384元/㎡
1555,"宝龙南苑 舒适两居室,温馨港湾等您来享",2室1厅 | 79.8㎡ |中层 （共12层） | 南向 | 2016年建,86万,10776元/㎡
1556,生态城精装洋房四室多一间老人房几代同堂有购房优惠房源,4室2厅 | 125㎡ |中层 （共7层） | 南北向 | 2020年建 |卓曼龙,100万,8000元/㎡


In [12]:

# 假设 df 是之前爬取到的数据
df_cleaned = df.copy()

# 1. 提取平方数（㎡）：从 "户型_面积等" 中匹配第一个 "|" 和第二个 "|" 之间的内容,去掉单位
def extract_square_meters(house_info):
    try:
        # 此处匹配 | 后面空格、数字（包括小数点）和"㎡"的模式
        match = re.search(r"\|\s*([\d.]+)㎡\s*\|", house_info)
        return float(match.group(1)) if match else None
    except Exception as e:
        return None

df_cleaned["平方数（㎡）"] = df_cleaned["户型_面积等"].apply(extract_square_meters)

# 2. 处理总价（万），去掉 "万" 单位
df_cleaned["总价（万）"] = df_cleaned["总价"].str.replace("万", "", regex=False).astype(float)

# 3. 处理单价（元/㎡），去掉 "元/㎡" 单位
df_cleaned["单价（元/㎡）"] = df_cleaned["单价"].str.replace("元/㎡", "", regex=False).astype(float)

# 4. 仅保留需要的三列
df_cleaned = df_cleaned[["平方数（㎡）", "总价（万）", "单价（元/㎡）"]]

# 将清洗后的 DataFrame 存储到 CSV 文件，不保存索引
df_cleaned.to_csv("zhongxin_selling.csv", index=False)

print("数据已保存")

数据已保存
